In [1]:
import pandas as pd
import numpy as np

# from api_data import ApiData
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser
from FUNCTIONS import get_current_location

In [ ]:
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

In [2]:
from api_route import ApiRoute
current_position = {'longitude': 130.844161,
                    'latitude': -12.456694}
final_position = {'longitude': 130.890346,
                  'latitude': -12.432393}
delta_spacing = 2000.0 # in meters
number_sites = 10
route = ApiRoute()
# dataframe = route.get_route_data(current_position)
dataframe = route.get_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing)
# dataframe = route.get_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites)

In [3]:
api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser)
api_executer = ApiExecuter(requester=api_requester)

Response status from siteinfo: OK.
Current sites' info has been retrieved. 
 Empty DataFrame
Columns: [name, site_id, longitude, latitude, altitude]
Index: []


In [ ]:
# api_executer.add_sites(route.route_data)
print(number_sites)

In [ ]:
print(api_requester.forecast_sites)

In [4]:
api_executer.add_sites(dataframe, print_is_requested=True)
# api_executer.delete_all_sites()

Response status from siteadd: OK.
Add site has been saved: 
   name site_id  longitude  latitude altitude
0    0  585259    130.844  -12.4567       32
Site with name 0 has been added in the API.
Response status from siteadd: OK.
Add site has been saved: 
   name site_id  longitude  latitude altitude
0    0  585259    130.844  -12.4567       32
1    1  585260    130.849  -12.4401       16
Site with name 1 has been added in the API.
Response status from siteadd: OK.
Add site has been saved: 
   name site_id  longitude  latitude altitude
0    0  585259    130.844  -12.4567       32
1    1  585260    130.849  -12.4401       16
2    2  585261    130.865  -12.4325       14
Site with name 2 has been added in the API.
Response status from siteadd: OK.
Add site has been saved: 
   name site_id  longitude  latitude altitude
0    0  585259    130.844  -12.4567       32
1    1  585260    130.849  -12.4401       16
2    2  585261    130.865  -12.4325       14
3    3  585262    130.883  -12.4324    

In [ ]:
position = {
    'latitude': -13.0,
    'longitude': 134.0
}
api_requester.get_site_edit(584978, True, name="aa")
api_requester.get_site_edit(584979, True, position=position)
api_requester.get_site_edit(584980, True, name="df", position=position)

In [5]:
forecasts = api_requester.get_solar_forecast()
# forecasts = api_requester.get_solar_forecast_cloudmove()
print(forecasts)

Response status from getforecast: OK.


AttributeError: 'list' object has no attribute 'items'

In [ ]:
index = forecasts.index

times = index.get_level_values('time').hour
print(times)

# Access the values of time and site_id
time_values = index.get_level_values('time')
site_id_values = index.get_level_values('site_id')

# Print the values
print("Time values:", time_values)
print("Site ID values:", site_id_values)

In [ ]:
# Extract all 'rr' values for all site_id and time combinations
# rr_values = forecasts.loc[:, ('rr', slice(None), slice(None))].droplevel('variable')
rr_values = forecasts.loc[:, ('rh')]
rr_values = rr_values.to_frame()
print(rr_values)
print(rr_values.max(), rr_values.min())
# # Assuming your MultiIndex DataFrame is named "forecast_data"
# pivot_table = rr_values.pivot(index='time', columns='site_id', values='tt')
# print(pivot_table)
rr_values.to_csv('rh_table.csv')

# print(rr_values)